In [4]:
from pathlib import Path
import re

import pandas as pd
import numpy as np

from pynwb import NWBHDF5IO
from fsspec import filesystem
from h5py import File
from dandi.dandiapi import DandiAPIClient

DANDISET_ID = '000559'
file_path = 'sub-HEK293/sub-HEK293_ses-exp3_ophys.nwb'

with DandiAPIClient() as client:
    asset = client.get_dandiset(DANDISET_ID, 'draft').get_asset_by_path(file_path)
    s3_url = asset.get_content_url(follow_redirects=1, strip_query=True)
fs = filesystem("http")
file_system = fs.open(s3_url, "rb")
file = File(file_system, mode="r")

with NWBHDF5IO(file=file, load_namespaces=True) as io:
    nwbfile = io.read()
nwbfile

root pynwb.file.NWBFile at 0x140208660583168
Fields:
  acquisition: {
    Images <class 'pynwb.base.Images'>,
    Reference1PSeries <class 'pynwb.ophys.OnePhotonSeries'>,
    Signal1PSeries <class 'pynwb.ophys.OnePhotonSeries'>
  }
  devices: {
    Microscope <class 'pynwb.device.Device'>
  }
  experiment_description: Abstract -- Spontaneous animal behaviour is built from action modules that are concatenated by the brain into sequences1,2. However, the neural mechanisms that guide the composition of naturalistic, self-motivated behaviour remain unknown. Here we show that dopamine systematically fluctuates in the dorsolateral striatum (DLS) as mice spontaneously express sub-second behavioural modules, despite the absence of task structure, sensory cues or exogenous reward. Photometric recordings and calibrated closed-loop optogenetic manipulations during open field behaviour demonstrate that DLS dopamine fluctuations increase sequence variation over seconds, reinforce the use of associa